In [1]:
import torch
import dhg
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

class GCNConv(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        bias: bool = True,
        drop_rate: float = 0.5,
    ):
        super().__init__()
        self.act = nn.ReLU(inplace=True)
        self.drop = nn.Dropout(drop_rate)
        self.theta = nn.Linear(in_channels, out_channels, bias=bias)

    def forward(self, X: torch.Tensor, g: dhg.Graph) -> torch.Tensor:
        if not isinstance(g, dhg.Graph):
            raise TypeError(f"Expected dhg.Graph, got {type(g)}")  
        X = self.theta(X)
        X_ = g.smoothing_with_GCN(X)
        X_ = self.drop(self.act(X_))
        return X_

class MiniGCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MiniGCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x
    

In [2]:
def train(model, data, optimizer=None, criterion=None):
    model.train()

    # Initialize optimizer & loss function if not provided
    if optimizer is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss()

    optimizer.zero_grad()
    out = model(data['x'], data['graph'])  # Forward pass
    
    # Compute loss
    loss = criterion(out[data['train_mask']], data['y'][data['train_mask']])
    
    # Backpropagation
    loss.backward()
    optimizer.step()

    # Compute accuracy
    pred = out.argmax(dim=1)  # Get predicted class indices
    correct = (pred[data['train_mask']] == data['y'][data['train_mask']]).sum().item()
    accuracy = correct / data['train_mask'].sum().item()

    return loss.item(), accuracy


def evaluate(model, data, criterion=None):
    model.eval()
    if criterion is None:
        criterion = torch.nn.CrossEntropyLoss()

    with torch.no_grad():
        out = model(data['x'], data['graph'])  # Forward pass
        loss = criterion(out[data['test_mask']], data['y'][data['test_mask']])

        pred = out.argmax(dim=1)
        correct = (pred[data['test_mask']] == data['y'][data['test_mask']]).sum().item()
        accuracy = correct / data['test_mask'].sum().item()

    return loss.item(), accuracy

In [3]:
%run data.ipynb

PATH_TO_IMAGE = "E:\MINI_PROJECT\PAVIA_UNI\PaviaU.mat"
PATH_TO_LABEL = "E:\MINI_PROJECT\PAVIA_UNI\PaviaU_gt.mat"

data = DATA(PATH_TO_IMAGE, PATH_TO_LABEL)

Creating graph with 207400 nodes and 1653504 edges.


In [7]:
graph_data = data.get_data_for_gcn()

# Now, you can use it with your GCN model
x = graph_data['x']
graph = graph_data['graph']  # Use 'graph' instead of 'edge_index'
y = graph_data['y']

# Example Model Usage
model = MiniGCN(input_dim=x.shape[1], hidden_dim=16, output_dim=y.max().item() + 1)

# Example Training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [8]:
num_epochs = 50
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, graph_data, optimizer, criterion)
    test_loss, test_acc = evaluate(model, graph_data, criterion)

    print(f"Epoch {epoch+1}/{num_epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

E:\Anaconda\Lib\site-packages\dhg\structure\graphs\graph.py:633: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  self.cache["L_GCN"] = _tmp_g.D_v_neg_1_2.mm(_tmp_g.A).mm(_tmp_g.D_v_neg_1_2).clone().coalesce()


Epoch 1/50 - Train Loss: 302.2103, Train Acc: 0.2302 | Test Loss: 253.4176, Test Acc: 0.0121
Epoch 2/50 - Train Loss: 534.7989, Train Acc: 0.1018 | Test Loss: 189.7412, Test Acc: 0.7938
Epoch 3/50 - Train Loss: 439.4505, Train Acc: 0.4759 | Test Loss: 162.3442, Test Acc: 0.7938
Epoch 4/50 - Train Loss: 252.9700, Train Acc: 0.5935 | Test Loss: 27.4402, Test Acc: 0.7919
Epoch 5/50 - Train Loss: 76.7698, Train Acc: 0.6877 | Test Loss: 98.9066, Test Acc: 0.1333
Epoch 6/50 - Train Loss: 160.7831, Train Acc: 0.5182 | Test Loss: 2.3026, Test Acc: 0.7938
Epoch 7/50 - Train Loss: 63.3123, Train Acc: 0.6479 | Test Loss: 2.3026, Test Acc: 0.7938
Epoch 8/50 - Train Loss: 27.7683, Train Acc: 0.7420 | Test Loss: 2.3026, Test Acc: 0.7938
Epoch 9/50 - Train Loss: 20.3241, Train Acc: 0.7684 | Test Loss: 2.3026, Test Acc: 0.7938
Epoch 10/50 - Train Loss: 18.9836, Train Acc: 0.7807 | Test Loss: 2.3026, Test Acc: 0.7938
Epoch 11/50 - Train Loss: 19.2047, Train Acc: 0.7846 | Test Loss: 2.3026, Test Acc: 0.